This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
from collections import defaultdict
import requests
from functools import reduce

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
response = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-01-01')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(response.json())
#column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'],

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2019-04-18T22:30:39.683Z', 'newest_available_date': '2019-04-18', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-01-01', 'end_date': '2017-01-01', 'data': [], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?" + \
      "&start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
json = requests.get(url).json()

Convert the returned JSON object into a Python dictionary.

In [6]:
_map = defaultdict(list)
for key in json['dataset_data']['data']:
    _map[key[0]] = key[1:]


Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
#highest = reduce(lambda x,y: x if map.get(y) [0] != None and map.get(x)[0] >= map.get(y)[0] else y, map, '2017-01-05')   
#print("the highest stock is " + str(map.get(highest)[0]) + " on " + highest)
highest = -999
for key,value in _map.items():
    if(value[0] != None):
        highest = max(highest, value[0])
print("The highest opening price is %.2f" % (highest))
    

The highest opening price is 53.11


In [8]:

#lowest = reduce(lambda x,y: x if map.get(y)[0] != None and map.get(x)[0] <= map.get(y)[0] else y, map)   
#print("the lowest stock is " + str(map.get(lowest)[2]) + " on " + lowest)

lowest = float("inf")
for key,value in _map.items():
    if(value[0] != None):
        lowest = min(lowest, value[0])
print("The lowest opening price is %.2f" % (lowest))


The lowest opening price is 34.00


What was the largest change in any one day (based on High and Low price)?

In [9]:
largestDelta = 0
largestDeltaDate = None
for key, value in _map.items():
    if abs(value[1] - value[2]) >= largestDelta:
        largestDelta = abs(value[1] - value[2])
        largestDeltaDate = key
print('The largest on any date is %.2f on %s' % (largestDelta, largestDeltaDate))


The largest on any date is 2.81 on 2017-05-11


What was the largest change between any two days (based on Closing Price)

In [10]:
largestDelta = 0
previousClose = float('inf')

for key,value in _map.items():
    if value[3] - previousClose > largestDelta:
        largestDelta = value[3] - previousClose
    previousClose = value[3]
print('The largest change between any 2 days is %.2f' % (largestDelta))

The largest change between any 2 days is 2.56


What was the average daily trading volume during this year?

In [11]:
total = 0
for value in _map.values():
    total += value[5]
print('The avg  daily trading volume is %.2f' % (total/len(_map.values())))
    

The avg  daily trading volume is 89124.34


(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
volumes = sorted([value[5] for value in _map.values()])

if len(_map) %2 != 0: 
  print("The median is %.2f" % volumes[len(volumes)//2])
else:
  print("The median is %.2f" % (volumes[len(volumes)//2 -1] + volumes[len(volumes)//2])/2) 


The median is 76286.00
